In [31]:
!{__import__('sys').executable} -m pip install --quiet -r requirements.txt

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [1]:
session_storage = 'downloads'

import os
os.makedirs(session_storage, exist_ok=True)

In [2]:
import fsspec

fs_dataset = fsspec.filesystem(
    'simplecache', 
    target_protocol='https', 
    cache_storage=session_storage
)

In [7]:
import pandas as pd
'''
import swifter
swifter.set_defaults(
    npartitions=None,
    dask_thres=1,
    scheduler='processes',
    progress_bar=True,
    progress_bar_desc=None,
    allow_dask_on_strings=True,
    force_parallel=False
)
'''

import sklearn as skl
import sklearn.base
import sklearn.compose
#import sklearn.ensemble
import sklearn.feature_extraction
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
import sklearn.pipeline
import sklearn.svm

import xgboost

import numpy as np

In [8]:
import sklearnex
sklearnex.patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Dataset

In [18]:
# TODO

dataset_lyrics = None
with fs_dataset.open('https://github.com/cse158-fa22-team-pushystrokers/datasets/raw/main/dataset-lyrics.csv.gz') as f:
    dataset_lyrics = (
        pd.read_csv(
            f, 
            compression='gzip'
        ).astype({
            'artist': 'category',
            'year': 'datetime64',
            'genre': 'category'
        }, copy=False)
    )

In [20]:
dataset_lyrics

,song,year,artist,genre,lyrics
0,ego-remix,2009-01-01,beyonce-knowles,Pop,"Oh baby, how you doing? You know I'm gonna cut..."
1,then-tell-me,2009-01-01,beyonce-knowles,Pop,"playin' everything so easy, it's like you seem..."
2,honesty,2009-01-01,beyonce-knowles,Pop,If you search For tenderness It isn't hard to ...
3,you-are-my-rock,2009-01-01,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I [Verse 1:] If I wrote a..."
4,black-culture,2009-01-01,beyonce-knowles,Pop,"Party the people, the people the party it's po..."
...,...,...,...,...,...
235989,who-am-i-drinking-tonight,2012-01-01,edens-edge,Country,"I gotta say Boy, after only just a couple of d..."
235990,liar,2012-01-01,edens-edge,Country,I helped you find her diamond ring You made me...
235991,last-supper,2012-01-01,edens-edge,Country,Look at the couple in the corner booth Looks a...
235992,christ-alone-live-in-studio,2012-01-01,edens-edge,Country,When I fly off this mortal earth And I'm measu...


In [32]:
dataset_lyrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235994 entries, 0 to 235993
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   song    235993 non-null  object        
 1   year    235994 non-null  datetime64[ns]
 2   artist  235994 non-null  category      
 3   genre   235994 non-null  category      
 4   lyrics  235994 non-null  object        
dtypes: category(2), datetime64[ns](1), object(2)
memory usage: 6.7+ MB


In [26]:
# TODO
# example
dataset_lyrics[dataset_lyrics['artist'] == 'florida-georgia-line']

,song,year,artist,genre,lyrics
114040,take-it-out-on-me,2013-01-01,florida-georgia-line,Country,What'd he do this time? Did he break your hear...
114041,hands-on-you,2013-01-01,florida-georgia-line,Country,"When I get my hands on you, I'm gonna do every..."
114042,headphones,2013-01-01,florida-georgia-line,Country,You were a whip around town Tinted windows dow...
114043,people-back-home,2013-01-01,florida-georgia-line,Country,This is for my people back home We're lighting...
114044,cruise-remix,2013-01-01,florida-georgia-line,Country,Baby you a song You make me wanna roll my wind...
114045,this-is-how-we-roll,2013-01-01,florida-georgia-line,Country,"The mix tape's got a little Hank, little Drake..."
114046,tell-me-how-you-like-it,2012-01-01,florida-georgia-line,Country,Do you like it when the meter's jumpin'? You l...
114047,party-people,2012-01-01,florida-georgia-line,Country,Hey hey hey hey Where my party people Oh oh oh...
114048,dayum-baby,2012-01-01,florida-georgia-line,Country,"Hey baby, listen you got me twistin', watching..."
114049,round-here,2012-01-01,florida-georgia-line,Country,"A hammer and a nail, stacking them bails I'm d..."


## EDA

## Predictive Task

## Model